# ARIMA-GARCH Modelling Test Script

In [4]:
import pandas as pd
from src import DataProcessor as DP
from src.ARIMAGARCHPipeline import ARIMAGARCHPipeline as AGP
import covcomp
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Initialize and load all your processed data
dp = DP.DataProcessor(secrets_path='../secrets/secretstest.json')

# Build the “input dict” that covcomp expects:
#    key = ticker, value = DataFrame with exactly ['Open Time','Covariance-LogDiff']
cov_input = {
    t: dp.crypto_data[t][['Open Time','Covariance-LogDiff']]
    for t in dp.tickers
}

# Compute the shrinkage‑adjusted covariance matrix:
#    You can tune `shrinkage` between 0 (no shrink) and 1 (full identity).
cov_dict = covcomp.compute_covariance(cov_input, shrinkage=0.05)

# Turn it into a DataFrame
cov_df = pd.DataFrame.from_dict(
    { t: dict(cov_dict[t]) for t in cov_dict }, 
    orient='index'
)
print("Shrinkage‑adjusted covariance matrix:\n", cov_df)

secrets.json file found at ../secrets/secretstest.json. Beginning initialization of DataProcessor class.
Binance client initialized successfully.
All required fields loaded successfully from secrets.json.
2 tickers loaded successfully.
Frequency: Daily
Starting date: 2015-01-01
Ending date: 2025-01-25
Base currency: USDT
Tickers: ['BTC', 'ETH']
Initialization of DataProcessor class completed successfully.


Fetching Crypto Data: 100%|██████████████████████████████| 2/2 [00:00<00:00]


Data for BTCUSDT already exists at /Users/devrajkatkoria/Documents/CryptoXQuant-Alpha/data/BTCUSDT.csv. Skipping download.
Data for ETHUSDT already exists at /Users/devrajkatkoria/Documents/CryptoXQuant-Alpha/data/ETHUSDT.csv. Skipping download.
All crypto data and indicators fetched/computed successfully and transformed to be stationary.
Fetching the last 51 data points for live data usage.


Fetching Live Data: 100%|██████████████████████████████| 2/2 [00:00<00:00]

All live data fetched successfully.
Data processing completed successfully.
Shrinkage‑adjusted covariance matrix:
           BTCUSDT   ETHUSDT
BTCUSDT  0.001393  0.001277
ETHUSDT  0.001277  0.002148


In [6]:
results = {}

for ticker, df in cov_input.items():
    # instantiate the pipeline for this single-asset series
    pipeline = AGP(df)
    out = pipeline.run()
    results[ticker] = out
    print(f"\n== {ticker} ==")
    print("  Best orders:",    out['arima_order'], out['garch_order'])
    print("  AICs:",           out['arima_aic'], out['garch_aic'])
    print("  Test MSE:",       out['test_mse'])
    print("  1σ coverage:",    out['1sigma_coverage'])


== BTCUSDT ==
  Best orders: (2, 0, 0) (1, 1)
  AICs: -8606.666241642497 -9458.487544258216
  Test MSE: 0.0007601042147540543
  1σ coverage: 0.8454106280193237

== ETHUSDT ==
  Best orders: (0, 0, 2) (1, 2)
  AICs: -7508.203629196962 -8224.5795648515
  Test MSE: 0.001276531139702745
  1σ coverage: 0.8442822384428224


- Fit quality (AIC): good—orders are parsimonious and AICs are well below zero.
- Point‐forecast accuracy (MSE): reasonable for ∼3 % daily moves.
- Risk‐forecast calibration (coverage): too conservative at ±1 σ.
